In [ ]:
#http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html#h2odeeplearningestimator
import h2o
import pandas as pd
from h2o.estimators import H2ODeepLearningEstimator
from statistics import mean
import matplotlib.pyplot as plt
import time

#h2o.cluster().shutdown()
h2o.init()
#h2o.cluster().show_status();
irisData = h2o.import_file("/Users/sukhwindersingh/Documents/study/sem2/PDC/iris/iris.data");
votesData = h2o.import_file("/Users/sukhwindersingh/Documents/study/sem2/PDC/votes/house-votes-84.data");
sonarData = h2o.import_file("/Users/sukhwindersingh/Documents/study/sem2/PDC/sonar/sonar.all-data");
#irisData.describe(chunk_summary=True);
#sonarData.describe(chunk_summary=True);
#votesData.describe(chunk_summary=True);

In [ ]:
print ('All  :', votesData.col_names)
print ('P  :', votesData.col_names[1:16])
print ('R  :', votesData.col_names[0])

In [ ]:
print ('ALL  :', irisData.col_names)
print ('P  :', irisData.col_names[0:4])
print ('R  :', irisData.col_names[4])
print ('----------------------------------')
print ('All  :', votesData.col_names)
print ('P  :', votesData.col_names[1:16])
print ('R  :', votesData.col_names[0])
print ('----------------------------------')
print ('All  :', sonarData.columns)
print ('P  :', sonarData.columns[:-1])
print ('R  :', sonarData.columns[60])


In [ ]:

    
Dict = {}
# Adding elements one at a time
Dict[0] = [0.15077023562772254, 0.21055075694250958]
Dict[1] = [0.9732789440345223, 0.027027027027026973]
Dict[2] = [5,5,5,5]  
print('fff',dictAvg(Dict))

In [ ]:
def dictAvg(d_in):
    d ={}
    for k in d_in.keys():
        d[k]=mean(d_in[k])
        #d[k]=sum(d_in[k]) / len(d_in[k])
    return d

def calculateMPCE(dataSet,predictorColumns,responseColumn):
    print ('P  :', predictorColumns)
    print ('R  :', responseColumn)
    map_train_error={}
    map_test_error={}
    list_train_error=[]
    list_test_error=[]
    activations =["rectifier", "tanh"]
    folds = [0,5,10]
    for activation in activations:    
        for fold in folds:
            string_model = activation  +"_"+  str(fold)
            map_train_error[string_model]=[];
            map_test_error[string_model]=[];
            for i in range(0,1):      
                dl = H2ODeepLearningEstimator(hidden=[5,20,100],
                                           epochs=10,
                                           activation=activation,
                                           nfolds=fold)
                if fold == 0:
                    trainData, testData = dataSet.split_frame(ratios=[.8])
                    t0 = time.time()
                    dl.train(x=predictorColumns,
                              y=responseColumn,
                              training_frame=trainData,
                              validation_frame=testData)
                    t0 = time.time() - t0
                    map_test_error[string_model].append(dl.mean_per_class_error(valid=True))
                    list_test_error.append(dl.mean_per_class_error(valid=True))
                else:
                    t0 = time.time()
                    dl.train(x=predictorColumns,
                         y=responseColumn,
                         training_frame=dataSet)
                    t0 = time.time() - t0
                    list_test_error.append(dl.mean_per_class_error(xval=True))
                    map_test_error[string_model].append(dl.mean_per_class_error(xval=True))
                list_train_error.append(dl.mean_per_class_error(train=True))
                map_train_error[string_model].append(dl.mean_per_class_error(train=True))
    #print('Map_Train_error :',map_train_error)
    #print('Map_Test_error :',map_test_error)
    map_train_error=dictAvg(map_train_error)
    map_test_error=dictAvg(map_test_error)
    print('Map_Train_error :',map_train_error)
    print('Map_Test_error :',map_test_error)
    #
    rect_list=[]
    tanh_list=[]
    for k, v in map_test_error.items():
        if(k.startswith('rectifier')):
            rect_list.append(v)
        else:
            tanh_list.append(v)
            
    #
    xpoints_1 = folds
    ypoints_1 = list(rect_list)
    xpoints_2 = folds
    ypoints_2 = list(tanh_list)
   
    plt.plot(xpoints_1, ypoints_1,label = "Rect",marker = '^',linestyle = 'dotted')
    plt.plot(xpoints_2, ypoints_2,label = "Tanh",marker = '*',linestyle = 'dotted')
    #plt.plot(xpoints_2, ypoints_2,label = "Test") 
    plt.legend()
    plt.xlabel('folds')
    plt.ylabel('classification_error[1]')
    #plt.title('NAMEEEE')
    plt.show()
    
    
calculateMPCE(irisData,irisData.col_names[0:4],irisData.col_names[4])
#calculateMSE(votesData,votesData.col_names[1:16],votesData.columns[0])
#calculateMSE(sonarData,sonarData.col_names[0:60],sonarData.col_names[60])

#SonarData Issue
#Time calculation

In [ ]:
#for seq in list(range(6)):
print(list(range(6)))
# Generate predictions on a test set (if necessary):
#pred = dl.predict(testIRIS)
#print(pred)
#print(dl.auc)
#print(dl.mse)
# print(irisData['class'].categories())
# print(irisData.col_names)
# print(len(irisData.col_names))
   # adaptive_rate=True
            #train_samples_per_iteration=-1,
            #reproducible=True,
            #single_node_mode=False,
            #balance_classes=False,
            #force_load_balance=False
            # Eval performance:
#perf = dl.model_performance(testIRIS)
#print(perf)
# list_iris= []
# list_iris.append(dl.mse(valid=True))
# list_iris.append(dl.mse(valid=True))
# print(list_iris)

In [ ]:
for i in range(5):
    print(i)


In [ ]:
    xpoints_1 = list(map_train_error.keys())
    ypoints_1 = list(map_train_error.values())
    xpoints_2 = list(map_test_error.keys())
    ypoints_2 = list(map_test_error.values())
   
    plt.plot(xpoints_1, ypoints_1,label = "Train")
    plt.plot(xpoints_2, ypoints_2,label = "Test") 
    plt.legend()
    plt.xlabel('activation_folds')
    plt.ylabel('classification_error[1]')
    #plt.title('NAMEEEE')
    plt.show()
    